#### Copyright 2020 Google LLC.

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Multiclass Classification

We previously created a binary classification model that determined if a piece of fruit was an orange or a grapefruit. There are many problems where binary classification can provide impactful solutions: spam or not spam in an email classifier, hit or hold in a blackjack simulation, buy or not in a stock market analysis. The list is basically endless.

There are other cases, however, where we want to make a decision across three or more classes. This is multiclass classification.

For many applications, multiclass classification can be broken down into many binary classification problems. These models employ a one-vs-all or one-vs-one strategy to create many binary classification tasks that are then aggregated into a multiclass classification model. Neural networks, decision trees, and k-nearest neighbors models are all capable of performing multiclass classification directly.

## The Dataset

For this unit we are going to use a classic machine learning dataset, the [Iris flower dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set). This is a dataset that was used in 1936 by British biologist and statistician Ronald Fisher to classify iris flowers into one of three species based on four measurements:

- The length of the petals
- The width of the petals
- The length of the sepals (the green petal-looking bits that are found at the base of the petals)
- The width of the sepals

Conveniently, the iris dataset is built into the scikit-learn library, so it is readily available to us. Let's take a look:

In [ ]:
from sklearn import datasets

iris_bunch = datasets.load_iris()
iris_bunch.keys()

In [ ]:
iris_bunch['target']

In [ ]:
iris_bunch['target_names']

Scikit-learn datasets are usually delivered in the form of a dictionary-like object called a `Bunch`. This `Bunch` contains the following fields:

- *DESCR*: A string describing the dataset.
- *data*: An array containing the features we are using for classifying. In this case, it's the four measurements listed above for each of 150 plants.
- *feature_names*: Labels for the data.
- *filename*: the file that this data came from.
- *target*: the values that we are trying to classify these flowers into. In this case, since we are dealing with three species of iris, we use three numbers (0, 1 and 2) to identify each species.
- *target_names*: labels for the target values. In this case, 0 refers to the setosa species, 1 to the versicolor species, and 2 to the virginica species.

We'll create a list of columns that we'll use for our model.

In [ ]:
FEATURES = iris_bunch['feature_names']
TARGET = 'species'

FEATURES, TARGET

Next we will load the feature and target data into a Pandas dataframe.

In [ ]:
import pandas as pd

iris_df = pd.DataFrame(iris_bunch['data'], columns=FEATURES)
iris_df[TARGET] = iris_bunch['target']

iris_df.sample(10)

Let's take a look at a description of the data.

In [ ]:
iris_df.describe()

There are 150 data points. No columns seem to be missing data and no values seem to be too far out of expected ranges. For example, there are no zero or negative lengths or widths, and the length and width values fall well within what we'd expect for a tulip.

We are interested in using the measurement features to predict the species of an iris. Let's take a closer look at the values we'll be predicting.

In this case we'll group by our 'species' feature and get a count of each species in our dataset.

In [ ]:
iris_df.groupby(TARGET).agg('count')

We have 50 examples of each species of iris, and overall we have only 150 samples. This presents two challenges. First, we don't have much data to actually build a model from. Second, the data that we do have is evenly distributed over class types. We might want to make sure that we train over the same distribution.

Luckily, there are solutions to both of these issues!

When we have data that has some weighted distribution across classes, we can do a **stratified split** to ensure that every class appears proportionally in our training data.

When we don't have enough data to properly train a model and don't feel that we can pull training data away, we can do a **k-fold cross validation** in order to utilize all of our data for training, while still trying to minimize model overfitting.

## Stratified Split

Let's first split off a set of data to use for our final model testing. We can use scikit-learn's `train_test_split` function to do this.

Since we have so little data, we'll only hold out 10% of the data for the final test.

After we make the split, we can see how many data points we will train off of for each class.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    iris_df[FEATURES],
    iris_df[TARGET],
    test_size=0.1,
    random_state=45)

y_train.groupby(y_train).count()

Yikes! We kept 50 data points for training class 1. That means we left none for final testing:

In [ ]:
y_test.groupby(y_test).count()

### Exercise 1: Stratified Train Test Split

We risk not holding out a data point for every class if we don't stratify our train test split. Rewrite the split above to create a stratified split. (If you don't remember how, try looking at the [documentation for `train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) and finding the argument that can be used to stratify the data.) When you are done, there should be 45 data points for each class in the training data and five data points for each class in the testing data. Print the counts to verify.

#### **Student Solution**

In [ ]:
# Your Code Goes Here
X_train, X_test, y_train, y_test = train_test_split(
    iris_df[FEATURES],
    iris_df[TARGET],
    test_size=0.2,
    stratify=iris_df[TARGET])

print(y_train.groupby(y_train).count())
print(y_test.groupby(y_test).count())

---

## Cross-Validation

Another problem we have is that we have very little data to work with. We only had 150 data points in total and are only going to train using 135 of those data points. If we are going to be hyperparameter tuning, we'll need a test and validation holdout, which will leave us very little data to train on.

One way to get around this is to use cross-validation. Cross-validation splits the data into a fixed number of tranches and trains on `n-1` of the tranches. Then it calculates a score using the holdout tranche. It does this repeatedly, holding out one tranche of data for each training pass. By looking at the mean of the scores for each training pass, you can get an idea of how well your model performs without having to specify a test dataset.

The `cross_val_score` method is used to perform the cross-validation. In the example below, we divide the data into five tranches and get five scores.

Since we are cross-validating with a classifier, scikit-learn automatically performs stratified splits for us.

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score

estimator = SGDClassifier()

scores = cross_val_score(
    estimator,
    X_train,
    y_train,
    cv=5
)

scores

We can now find the mean score.

In [ ]:
scores.mean()

What does this score represent, though? It turns out that it uses the default scoring method for the classifier that we used. In this case we used the [`SGDClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html), which reports accuracy by default.

Also note that the estimator isn't trained after running cross-validation. You can run cross-validation to test different data preprocessing pipelines and hyperparameters. Once you are happy with a specific setup, you'll need to train the model with the chosen pipeline and parameters.

### Exercise 2: F1 Scoring

What if we wanted to use F1 for our scoring metric instead of accuracy?

Run `cross_val_score` on an `SGDClassifier` and get the F1 score. Check out the documentation for [`cross_val_score`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html), [`make_scorer`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html), and [`f1_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html) for clues.

#### **Student Solution**

In [ ]:
# Your Code Goes here
import pandas as pd

from sklearn import datasets
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

iris_bunch = datasets.load_iris()
iris_bunch.keys()

FEATURES = iris_bunch['feature_names']
TARGET = 'species'

iris_df = pd.DataFrame(iris_bunch['data'], columns=FEATURES)
iris_df[TARGET] = iris_bunch['target']

X_train, X_test, y_train, y_test = train_test_split(
    iris_df[FEATURES],
    iris_df[TARGET], 
    test_size=0.1,
    stratify=iris_df[TARGET])

# Solution Starts Here

estimator = SGDClassifier()

scores = cross_val_score(
    estimator,
    X_train[FEATURES],
    y_train,
    cv=5,
    scoring=make_scorer(f1_score, average='micro')
)


scores.mean()

---

# The Model Pipeline

Since we are now using cross-validation to train the model, we can use our testing holdout data as a final validation. Let's make that clear by renaming the data.

In [ ]:
X_validation = X_test
y_validation = y_test

Now we can work on tuning the model and the model pipeline.

Let's first look back at the data going into the model:



In [ ]:
iris_df[FEATURES].describe()

The data is all in the same order of magnitude, but columns like 'sepal length (cm)' are considerably larger than columns like 'petal width (cm)'.

We need to perform some preprocessing to get the data into a more uniform shape before feeding it to the model. To do that we'll use the [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html), which removes the mean and subtracts the unit variance from each column of data.

To use the `StandardScaler`, we create the object, `fit()` the data, and then `transform()`.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(iris_df[FEATURES])

pd.DataFrame(
    scaler.transform(iris_df[FEATURES]),
    columns=FEATURES
).describe()

You can see in the output of `describe()` that the data now all has a standard deviation that approaches one.

We need to perform this preprocessing to features before training the model and before getting predictions. It can be error-prone to try to remember to do this. To make the task easier, we can create an estimator [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) that applies our transformations and calls our estimator.

In [ ]:
from sklearn.pipeline import Pipeline

estimator = Pipeline(
  steps=[
    ['scale', StandardScaler()],
    ['classifier', SGDClassifier()],
  ]
)

scores = cross_val_score(
    estimator,
    X_train[FEATURES],
    y_train,
    cv=5,
)

scores.mean()

Scaling gave us a considerable jump in accuracy score. Hopefully you see similar results.

### Exercise 3: Final Validation

Our accuracy results were pretty good, so we aren't going to do any more hyperparameter tuning in this lab. Before we declare victory, though, we should find the F1 score of our validation data. Using our estimator pipeline, calculate the F1 score for `X_validation`.

In [ ]:
# Your Code Goes Here
import pandas as pd

from sklearn import datasets
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

iris_bunch = datasets.load_iris()
iris_bunch.keys()

FEATURES = iris_bunch['feature_names']
TARGET = 'species'

iris_df = pd.DataFrame(iris_bunch['data'], columns=FEATURES)
iris_df[TARGET] = iris_bunch['target']

X_train, X_validation, y_train, y_validation = train_test_split(
    iris_df[FEATURES],
    iris_df[TARGET], 
    test_size=0.1,
    stratify=iris_df[TARGET])

estimator = Pipeline(
  steps=[
    ['scale', StandardScaler()],
    ['classifier', SGDClassifier()],
  ]
)

scores = cross_val_score(
    estimator,
    X_train[FEATURES],
    y_train,
    cv=5,
    # We calculated F1 here too. This isn't required.
    scoring=make_scorer(f1_score, average='micro')
)

# Solution Starts Here
estimator.fit(X_train, y_train)
predictions = estimator.predict(X_validation)
f1_score(y_validation, predictions, average='micro')

In [ ]:
#importing confusion matrix
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_validation, predictions)
print('Confusion Matrix\n')
print(confusion)

#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_validation, predictions)))

print('Micro Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_validation, predictions, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_validation, predictions, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_validation, predictions, average='weighted')))

from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_validation, predictions, target_names=['Class 1', 'Class 2', 'Class 3']))

---

# Exercise 4: Winemaker Identification

Scikit-learn comes prepackaged with many toy datasets. These can be found in the [`sklearn.datasets` package](https://scikit-learn.org/stable/datasets/index.html). In this exercise we'll be working with the [wine dataset](https://scikit-learn.org/stable/datasets/index.html#wine-dataset).

The dataset contains information about the properties of wines produced by three different producers. The grapes that the producers used all come from the same region.

The columns are:

* alcohol
* malic_acid
* ash
* alcalinity_of_ash
* magnesium
* total_phenols
* flavanoids
* nonflavanoid_phenols
* proanthocyanins
* color_intensity
* hue
* od280/od315_of_diluted_wines
* proline

The target column is a 0, 1, or 2. Each number represents a different producer.

Your task in this exercise is to create a classifier that can identify the producer based on the wine properties.

Use as many code blocks as necessary to examine the data and build and validate your model. Document your process using text blocks and/or comments in your code.

**Student Solution**

In [1]:
# Your Code Goes Here
import pandas as pd

from sklearn import datasets
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

wine_bunch = datasets.load_wine()
wine_bunch.keys()


dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

In [2]:

FEATURES = wine_bunch['feature_names']
TARGET = 'species'

wine_df = pd.DataFrame(wine_bunch['data'], columns=FEATURES)
wine_df[TARGET] = wine_bunch['target']


In [3]:
wine_df.head(10)

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,species
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
5,14.20,1.76,2.45,15.2,112.0,3.27,3.39,0.34,1.97,6.75,1.05,2.85,1450.0,0
6,14.39,1.87,2.45,14.6,96.0,2.50,2.52,0.30,1.98,5.25,1.02,3.58,1290.0,0
7,14.06,2.15,2.61,17.6,121.0,2.60,2.51,0.31,1.25,5.05,1.06,3.58,1295.0,0
8,14.83,1.64,2.17,14.0,97.0,2.80,2.98,0.29,1.98,5.20,1.08,2.85,1045.0,0
9,13.86,1.35,2.27,16.0,98.0,2.98,3.15,0.22,1.85,7.22,1.01,3.55,1045.0,0


In [4]:
import pandas as pd

wine_df = pd.DataFrame(wine_bunch['data'], columns=FEATURES)
wine_df[TARGET] = wine_bunch['target']

wine_df.sample(10)

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,species
137,12.53,5.51,2.64,25.0,96.0,1.79,0.60,0.63,1.10,5.00,0.82,1.69,515.0,2
116,11.82,1.47,1.99,20.8,86.0,1.98,1.60,0.30,1.53,1.95,0.95,3.33,495.0,1
134,12.51,1.24,2.25,17.5,85.0,2.00,0.58,0.60,1.25,5.45,0.75,1.51,650.0,2
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
6,14.39,1.87,2.45,14.6,96.0,2.50,2.52,0.30,1.98,5.25,1.02,3.58,1290.0,0
165,13.73,4.36,2.26,22.5,88.0,1.28,0.47,0.52,1.15,6.62,0.78,1.75,520.0,2
167,12.82,3.37,2.30,19.5,88.0,1.48,0.66,0.40,0.97,10.26,0.72,1.75,685.0,2
39,14.22,3.99,2.51,13.2,128.0,3.00,3.04,0.20,2.08,5.10,0.89,3.53,760.0,0
153,13.23,3.30,2.28,18.5,98.0,1.80,0.83,0.61,1.87,10.52,0.56,1.51,675.0,2
110,11.46,3.74,1.82,19.5,107.0,3.18,2.58,0.24,3.58,2.90,0.75,2.81,562.0,1


In [5]:
wine_df.describe()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,species
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258,0.938202
std,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474,0.775035
min,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000,0.000000
25%,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000,0.000000
50%,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000,1.000000
75%,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000,2.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000,2.000000


In [6]:
wine_df.groupby(TARGET).agg('count')

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
species,,,,,,,,,,,,,
0,59,59,59,59,59,59,59,59,59,59,59,59,59
1,71,71,71,71,71,71,71,71,71,71,71,71,71
2,48,48,48,48,48,48,48,48,48,48,48,48,48


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    wine_df[FEATURES],
    wine_df[TARGET],
    test_size=0.1,
    random_state=45)

y_train.groupby(y_train).count()

species
0    54
1    62
2    44
Name: species, dtype: int64

In [8]:
y_test.groupby(y_test).count()

species
0    5
1    9
2    4
Name: species, dtype: int64

In [9]:
# Your Code Goes Here
X_train, X_test, y_train, y_test = train_test_split(
    wine_df[FEATURES],
    wine_df[TARGET],
    test_size=0.2,
    stratify=wine_df[TARGET])

print(y_train.groupby(y_train).count())
print(y_test.groupby(y_test).count())

species
0    47
1    57
2    38
Name: species, dtype: int64
species
0    12
1    14
2    10
Name: species, dtype: int64


In [10]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score

estimator = SGDClassifier()

scores = cross_val_score(
    estimator,
    X_train,
    y_train,
    cv=5
)

scores

array([0.5862069 , 0.48275862, 0.53571429, 0.67857143, 0.57142857])

In [11]:
scores.mean()

0.5709359605911329

In [12]:
# Your Code Goes here
import pandas as pd

from sklearn import datasets
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

wine_bunch = datasets.load_wine()
wine_bunch.keys()

FEATURES = wine_bunch['feature_names']
TARGET = 'species'

iris_df = pd.DataFrame(wine_bunch['data'], columns=FEATURES)
iris_df[TARGET] = wine_bunch['target']

X_train, X_test, y_train, y_test = train_test_split(
    iris_df[FEATURES],
    iris_df[TARGET], 
    test_size=0.1,
    stratify=iris_df[TARGET])

# Solution Starts Here

estimator = SGDClassifier()

scores = cross_val_score(
    estimator,
    X_train[FEATURES],
    y_train,
    cv=5,
    scoring=make_scorer(f1_score, average='micro')
)


scores.mean()

0.575

In [13]:
X_validation = X_test
y_validation = y_test

In [14]:
wine_df[FEATURES].describe()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258
std,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474
min,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000
25%,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000
50%,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000
75%,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000


In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(wine_df[FEATURES])

pd.DataFrame(
    scaler.transform(wine_df[FEATURES]),
    columns=FEATURES
).describe()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
count,1.780000e+02,1.780000e+02,1.780000e+02,1.780000e+02,1.780000e+02,1.780000e+02,1.780000e+02,1.780000e+02,1.780000e+02,1.780000e+02,1.780000e+02,1.780000e+02,1.780000e+02
mean,7.841418e-15,2.444986e-16,-4.059175e-15,-7.110417e-17,-2.494883e-17,-1.955365e-16,9.443133e-16,-4.178929e-16,-1.540590e-15,-4.129032e-16,1.398382e-15,2.126888e-15,-6.985673e-17
std,1.002821e+00,1.002821e+00,1.002821e+00,1.002821e+00,1.002821e+00,1.002821e+00,1.002821e+00,1.002821e+00,1.002821e+00,1.002821e+00,1.002821e+00,1.002821e+00,1.002821e+00
min,-2.434235e+00,-1.432983e+00,-3.679162e+00,-2.671018e+00,-2.088255e+00,-2.107246e+00,-1.695971e+00,-1.868234e+00,-2.069034e+00,-1.634288e+00,-2.094732e+00,-1.895054e+00,-1.493188e+00
25%,-7.882448e-01,-6.587486e-01,-5.721225e-01,-6.891372e-01,-8.244151e-01,-8.854682e-01,-8.275393e-01,-7.401412e-01,-5.972835e-01,-7.951025e-01,-7.675624e-01,-9.522483e-01,-7.846378e-01
50%,6.099988e-02,-4.231120e-01,-2.382132e-02,1.518295e-03,-1.222817e-01,9.595986e-02,1.061497e-01,-1.760948e-01,-6.289785e-02,-1.592246e-01,3.312687e-02,2.377348e-01,-2.337204e-01
75%,8.361286e-01,6.697929e-01,6.981085e-01,6.020883e-01,5.096384e-01,8.089974e-01,8.490851e-01,6.095413e-01,6.291754e-01,4.939560e-01,7.131644e-01,7.885875e-01,7.582494e-01
max,2.259772e+00,3.109192e+00,3.156325e+00,3.154511e+00,4.371372e+00,2.539515e+00,3.062832e+00,2.402403e+00,3.485073e+00,3.435432e+00,3.301694e+00,1.960915e+00,2.971473e+00


In [16]:
from sklearn.pipeline import Pipeline

estimator = Pipeline(
  steps=[
    ['scale', StandardScaler()],
    ['classifier', SGDClassifier()],
  ]
)

scores = cross_val_score(
    estimator,
    X_train[FEATURES],
    y_train,
    cv=5,
)

scores.mean()

0.98125

In [17]:
# Your Code Goes Here
import pandas as pd

from sklearn import datasets
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

wine_bunch = datasets.load_wine()
wine_bunch.keys()

FEATURES = wine_bunch['feature_names']
TARGET = 'species'

wine_df = pd.DataFrame(wine_bunch['data'], columns=FEATURES)
wine_df[TARGET] = wine_bunch['target']

X_train, X_validation, y_train, y_validation = train_test_split(
    wine_df[FEATURES],
    wine_df[TARGET], 
    test_size=0.1,
    stratify=wine_df[TARGET])

estimator = Pipeline(
  steps=[
    ['scale', StandardScaler()],
    ['classifier', SGDClassifier()],
  ]
)

scores = cross_val_score(
    estimator,
    X_train[FEATURES],
    y_train,
    cv=5,
    # We calculated F1 here too. This isn't required.
    scoring=make_scorer(f1_score, average='micro')
)

# Solution Starts Here
estimator.fit(X_train, y_train)
predictions = estimator.predict(X_validation)
f1_score(y_validation, predictions, average='micro')

0.9444444444444444

In [18]:
#importing confusion matrix
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_validation, predictions)
print('Confusion Matrix\n')
print(confusion)

#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_validation, predictions)))

print('Micro Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_validation, predictions, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_validation, predictions, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_validation, predictions, average='weighted')))

from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_validation, predictions, target_names=['Class 1', 'Class 2', 'Class 3']))

Confusion Matrix

[[5 1 0]
 [0 7 0]
 [0 0 5]]

Accuracy: 0.94

Micro Precision: 0.94
Micro Recall: 0.94
Micro F1-score: 0.94

Macro Precision: 0.96
Macro Recall: 0.94
Macro F1-score: 0.95

Weighted Precision: 0.95
Weighted Recall: 0.94
Weighted F1-score: 0.94

Classification Report

              precision    recall  f1-score   support

     Class 1       1.00      0.83      0.91         6
     Class 2       0.88      1.00      0.93         7
     Class 3       1.00      1.00      1.00         5

    accuracy                           0.94        18
   macro avg       0.96      0.94      0.95        18
weighted avg       0.95      0.94      0.94        18



---